In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
df = pd.read_csv('/kaggle/input/airlines-flights-data/airlines_flights_data.csv')
df1 = df.copy()
df.head()

In [ ]:
!pip install ydata-profiling
!pip install pydantic-settings
from pydantic_settings import BaseSettings
from ydata_profiling import ProfileReport

def profile_data(df: pd.DataFrame, output_file: str = None):
    """
    Generate an interactive data profiling report.
    """
    profile = ProfileReport(df, title="Dataset Profiling Report", explorative=True)
    if output_file:
        profile.to_file(output_file)
    else:
        profile.to_notebook_iframe()
    return profile

def auto_detect_problem_type(df: pd.DataFrame, target_col: str = None) -> str:
    """
    Automatically detect the ML problem type based on target column and dataset.
    """
    if target_col is None or target_col not in df.columns:
        return "unsupervised (clustering or other)"

    target = df[target_col]
    datetime_cols = df.select_dtypes(include=['datetime64[ns]', 'datetime64']).columns
    if pd.api.types.is_datetime64_any_dtype(df.index) or len(datetime_cols) > 0:
        if pd.api.types.is_numeric_dtype(target):
            return "time series regression"
        else:
            return "time series classification"

    if pd.api.types.is_numeric_dtype(target):
        unique_vals = target.nunique()
        if unique_vals > 20:
            return "regression"
        else:
            return "classification"

    if pd.api.types.is_categorical_dtype(target) or target.dtype == object:
        return "classification"

    return "unknown"

# Example usage
# DATA_PATH = "/kaggle/input/airlines-flights-data/airlines_flights_data.csv"
TARGET_COL = "duration"  # Replace with actual target column name

# df = pd.read_csv(DATA_PATH)

# Profile dataset (generates interactive report in notebook or saves to file)
profile = profile_data(df)  # For Jupyter notebook display
# profile = profile_data(df, output_file="data_profile_report.html")  # To save HTML report

# Auto detect problem type
problem_type = auto_detect_problem_type(df, TARGET_COL)
print(f"Auto-detected problem type: {problem_type}")



In [ ]:
# import pandas as pd
# Separate numerical and categorical columns
# Separate numerical and categorical columns
# num_cols = df.select_dtypes(include=['int64', 'float64']).columns
# cat_cols = df.select_dtypes(include=['object', 'category']).columns

# df1 = df.copy()
# Separate numerical and categorical columns
num_cols = df1.select_dtypes(include=['int64', 'float64']).columns
cat_cols = df1.select_dtypes(include=['object', 'category']).columns

def remove_duplicates(df1):
    """
    Remove duplicate rows from dataset.
    """
    before = len(df1)
    df1 = df1.drop_duplicates()
    after = len(df1)
    print(f"Duplicates removed: {before - after}")
    print("remove dups completed")
    return df1
remove_duplicates(df1)


def check_missing_values(df1):
    """
    Display missing value counts and percentages per column.
    """
    missing = df1.isnull().sum()
    percent = (missing / len(df1)) * 100
    missing_df = pd.DataFrame({'MissingCount': missing, 'MissingPercent': percent})
    print(missing_df[missing_df['MissingCount'] > 0])
check_missing_values(df1)

# Imputation example: mean for numerical, mode for categorical
def impute_missing_values(df1, num_cols, cat_cols):
    """
    Impute missing values in numerical and categorical columns.
    """
    for col in num_cols:
        mean_value = df1[col].mean()
        df1[col].fillna(mean_value, inplace=True)

    for col in cat_cols:
        mode_value = df1[col].mode()[0]
        df1[col].fillna(mode_value, inplace=True)
    return df1
impute_missing_values(df1, num_cols,cat_cols )


In [ ]:
df.isnull().sum()

In [ ]:
df1.isnull().sum()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# # Load dataset (replace with actual path)
# DATA_PATH = "data/dataset.csv"
# df = pd.read_csv(DATA_PATH)

# df = df.drop(columns = {"df_index"})

# Basic statistical summary
print(df1.describe(include='all'))

# Separate numerical and categorical columns
num_cols = df.select_dtypes(include=['int64', 'float64']).columns
cat_cols = df.select_dtypes(include=['object', 'category']).columns

# Histograms for numerical features
df[num_cols].hist(bins=20, figsize=(15, 10))
plt.suptitle("Numerical Feature Distributions")
plt.show()

# Boxplots for outlier visualization
for col in num_cols:
    plt.figure(figsize=(6, 3))
    sns.boxplot(x=df[col])
    plt.title(f"Boxplot of {col}")
    plt.show()

# Correlation heatmap (Pearson)
plt.figure(figsize=(12, 8))
sns.heatmap(df[num_cols].corr(), annot=True, cmap='coolwarm', fmt=".2f")
plt.title("Correlation Heatmap")
plt.show()

# Countplots for categorical features
for col in cat_cols:
    plt.figure(figsize=(8, 4))
    sns.countplot(data=df, x=col, order=df[col].value_counts().index)
    plt.title(f"Category Counts for {col}")
    plt.xticks(rotation=45)
    plt.show()

# Missing values heatmap
plt.figure(figsize=(10, 6))
sns.heatmap(df1.isnull(), cbar=False, yticklabels=False, cmap='viridis')
plt.title("Missing Values Heatmap")
plt.show()


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import shapiro

# Assuming 'df' is your DataFrame, already loaded.
# For example: df = pd.read_csv('your_data.csv')

# List of columns to check for normality
columns_to_check = ['duration', 'price']

# Create a figure for plotting
fig, axes = plt.subplots(1, len(columns_to_check), figsize=(12, 5))
fig.suptitle('Distribution of Key Columns', fontsize=16)

for i, col in enumerate(columns_to_check):
    print(f"\n--- Checking Normality for '{col}' ---")
    
    # Drop missing values for the statistical test
    data_to_test = df1[col].dropna()
    
    # Visual check with a histogram
    sns.histplot(data_to_test, kde=True, ax=axes[i])
    axes[i].set_title(f'Histogram of {col}')
    
    # Statistical check with Shapiro-Wilk test
    stat, p_value = shapiro(data_to_test)
    
    print(f"Shapiro-Wilk Test Statistic: {stat:.4f}")
    print(f"P-value: {p_value:.4f}")
    
    # Interpretation
    if p_value > 0.05:
        print(f"Conclusion: The '{col}' column likely comes from a normal distribution.")
    else:
        print(f"Conclusion: The '{col}' column likely does not come from a normal distribution.")

plt.tight_layout(rect=[0, 0, 1, 0.96])
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np

def plot_boxplots(data):
    """
    Plot boxplots for all numeric columns in a DataFrame.
    """
    # Find all numeric columns in the DataFrame
    numeric_cols = data.select_dtypes(include=np.number).columns.tolist()

    # Check if any numeric columns were found
    if not numeric_cols:
        print("No numeric columns found to plot.")
        return

    # Loop through the list of numeric column names
    for col in numeric_cols:
        plt.figure(figsize=(8, 4)) # Adjusted figure size for better readability
        sns.boxplot(x=data[col])
        plt.title(f"Boxplot of {col}")
        plt.show()

plot_boxplots(df1)

In [ ]:
df = df.drop('df_index', axis=1)

In [ ]:
#After Cleaning as well if any outliers use winsorize to cap. outliers correction only applicatble for num cols. 
#if col hav outlier is normal then apply zscore technique
#when not col data is not normal or skew then use below normal to find the outliers


# def detect_outliers_iqr(data, col):
#     """
#     Detect outliers in a column using the IQR method.
#     """
#     Q1 = data[col].quantile(0.25)
#     Q3 = data[col].quantile(0.75)
#     IQR = Q3 - Q1
#     lower_bound = Q1 - 1.5 * IQR
#     upper_bound = Q3 + 1.5 * IQR
#     outliers = data[(data[col] < lower_bound) | (data[col] > upper_bound)]
#     return outliers

# outliers_in_duration = detect_outliers_iqr(df1, 'duration')
# outliers_in_price = detect_outliers_iqr(df1, 'price')

# # Now you can print or view the results.
# print("Outliers in the 'duration' column:")
# print(outliers_in_duration)

# print("\nOutliers in the 'price' column:")
# print(outliers_in_price)

from scipy.stats import zscore
def detect_outliers_zscore(data, col, threshold=3):
    """
    Detect outliers in a column using the Z-score method.
    """
    # Create a Series with Z-scores and align its index with the original data
    z_scores_series = data[col].dropna()
    z_scores = zscore(z_scores_series)

    # Use the index of the Z-scores to filter the original DataFrame
    outlier_indices = z_scores_series[np.abs(z_scores) > threshold].index
    outliers = data.loc[outlier_indices]
    
    return outliers

detect_outliers_zscore(df, 'duration',threshold=3)


In [ ]:
# No outliers skipping winsorization to cap



numeric_cols = df1.select_dtypes(include=np.number).columns.tolist()
def winsorize_column(data, col):
    """
    Cap outliers using the IQR method bounds.
    """
    Q1 = data[col].quantile(0.25)
    Q3 = data[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    data[col] = np.where(data[col] < lower_bound, lower_bound, data[col])
    data[col] = np.where(data[col] > upper_bound, upper_bound, data[col])
    return df
winsorize_column(df1,numeric_cols)

In [ ]:
#Same here not outliers..skipping winsorization

from scipy.stats.mstats import winsorize
num_cols = df1.select_dtypes(include=['int64', 'float64']).columns

# Detect outliers for first numerical column using IQR
column_to_check = num_cols[0]
outliers_iqr = detect_outliers_zscore(df1, column_to_check)
print(f"Outliers detected by IQR in {column_to_check}:")
print(outliers_iqr)

# Visualize outliers
plot_boxplots(df1)

# Treat outliers with winsorization
df = winsorize_column(df1, column_to_check)
print(f"After winsorization, summary stats for {column_to_check}:")
print(df1[column_to_check].describe())


In [ ]:
df1.shape

In [ ]:
df.shape

In [ ]:
df1.head()

In [ ]:
df1.shape

In [ ]:
df1.sample(n=10, random_state=42)

In [ ]:
# import pandas as pd
# from sklearn.preprocessing import LabelEncoder

# def heuristic_auto_encode(df, ordinal_threshold=5):
#     """
#     Automatically applies Label Encoding or One-Hot Encoding based on a
#     cardinality threshold.
    
#     Args:
#         df (pd.DataFrame): The DataFrame to encode.
#         ordinal_threshold (int): The maximum number of unique categories
#                                  for a column to be considered for Label Encoding.
        
#     Returns:
#         pd.DataFrame: A new DataFrame with all categorical columns encoded.
#     """
#     encoded_df = df.copy()
    
#     all_cat_cols = encoded_df.select_dtypes(include=['object', 'category']).columns.tolist()
    
#     label_encode_cols = []
#     ohe_cols = []
    
#     for col in all_cat_cols:
#         if encoded_df[col].nunique() <= ordinal_threshold:
#             # Assume columns with few unique values are ordinal
#             label_encode_cols.append(col)
#         else:
#             # Assume columns with many unique values are nominal
#             ohe_cols.append(col)
            
#     # Apply Label Encoding
#     for col in label_encode_cols:
#         le = LabelEncoder()
#         encoded_df[col] = le.fit_transform(encoded_df[col])
#         print(f"Applied Label Encoding to: {col} (Unique values: {df[col].nunique()})")
    
#     # Apply One-Hot Encoding
#     encoded_df = pd.get_dummies(encoded_df, columns=ohe_cols, drop_first=True)
#     print(f"Applied One-Hot Encoding to: {ohe_cols}")
        
#     return encoded_df

# # --- How to use this function ---
# # Assuming df1 is your cleaned DataFrame
# df_encoded = heuristic_auto_encode(df1, ordinal_threshold=5)

# print("\nFirst 5 rows of the encoded DataFrame:")
# print(df_encoded.head())

In [ ]:
# Get the count of unique values in the 'flight' column
unique_flight_count = df1['flight'].nunique()

# Print the result
print(f"The 'flight' column has {unique_flight_count} unique values.")

# You can also check the total number of rows in the DataFrame for comparison
total_rows = len(df1)
print(f"The entire DataFrame has {total_rows} rows.")

In [ ]:
# if unique values are 95% in a clumn then we should not apply encoders to it
# Assuming your DataFrame is named 'df'
total_rows = len(df1)

# Check all columns and identify those with high cardinality
cols_to_exclude = []
for col in df1.columns:
    unique_ratio = df1[col].nunique() / total_rows
    if unique_ratio > 0.95:  # You can adjust this threshold
        cols_to_exclude.append(col)
        print(f"Column '{col}' has a unique ratio of {unique_ratio:.2f} and should likely be excluded.")

print(f"\nFinal list of columns to exclude: {cols_to_exclude}")

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
import category_encoders as ce

def magic_auto_encode(df, low_cardinality_threshold=5, high_cardinality_threshold=50):
    """
    Automatically applies different encoders based on the number of unique values.

    Args:
        df (pd.DataFrame): The DataFrame to encode.
        low_cardinality_threshold (int): Max unique values for Label Encoding.
        high_cardinality_threshold (int): Min unique values for Binary Encoding.

    Returns:
        pd.DataFrame: A new DataFrame with all categorical columns encoded.
    """
    encoded_df = df1.copy()
    
    # Identify all categorical columns
    all_cat_cols = encoded_df.select_dtypes(include=['object', 'category']).columns.tolist()
    label_encode_cols = []
    ohe_cols = []
    binary_encode_cols = []
    
    for col in all_cat_cols:
        n_unique = encoded_df[col].nunique()
        if n_unique <= low_cardinality_threshold:
            label_encode_cols.append(col)
        elif n_unique > high_cardinality_threshold:
            binary_encode_cols.append(col)
        else:
            ohe_cols.append(col)
    
    print("--- Encoding Summary ---")
    print(f"Applying Label Encoding to: {label_encode_cols}")
    print(f"Applying One-Hot Encoding to: {ohe_cols}")
    print(f"Applying Binary Encoding to: {binary_encode_cols}")
    
    # Apply Label Encoding
    for col in label_encode_cols:
        le = LabelEncoder()
        encoded_df[col] = le.fit_transform(encoded_df[col])
    
    # Apply One-Hot Encoding
    encoded_df = pd.get_dummies(encoded_df, columns=ohe_cols, drop_first=True)
    
    # Apply Binary Encoding
    if binary_encode_cols:
        encoder = ce.BinaryEncoder(cols=binary_encode_cols)
        encoded_df = encoder.fit_transform(encoded_df)
        
    return encoded_df

# --- How to use this function ---
# Assuming df1 is your cleaned DataFrame
df_encoded = magic_auto_encode(df1, low_cardinality_threshold=5, high_cardinality_threshold=50)

print("\nFirst 5 rows of the encoded DataFrame:")
print(df_encoded.head())

In [ ]:
# import pandas as pd
# import numpy as np
# from sklearn.preprocessing import LabelEncoder
# import category_encoders as ce

# def magic_auto_encode(df, low_cardinality_threshold=5, high_cardinality_threshold=50, high_cardinality_strategy='binary', exclude_cols=[]):
#     """
#     Automatically applies different encoders based on cardinality, with
#     an option for high-cardinality columns.

#     Args:
#         df (pd.DataFrame): The DataFrame to encode.
#         low_cardinality_threshold (int): Max unique values for Label Encoding.
#         high_cardinality_threshold (int): Min unique values for high-cardinality strategy.
#         high_cardinality_strategy (str): 'binary' or 'frequency'.
#         exclude_cols (list): A list of columns to exclude from encoding.

#     Returns:
#         pd.DataFrame: A new DataFrame with all categorical columns encoded.
#     """
#     encoded_df = df.copy()
    
#     all_cat_cols = [col for col in encoded_df.select_dtypes(include=['object', 'category']).columns.tolist() if col not in magic_auto_encode]
    
#     label_encode_cols = []
#     ohe_cols = []
#     high_card_cols = []
    
#     for col in all_cat_cols:
#         n_unique = encoded_df[col].nunique()
#         if n_unique <= low_cardinality_threshold:
#             label_encode_cols.append(col)
#         elif n_unique > high_cardinality_threshold:
#             high_card_cols.append(col)
#         else:
#             ohe_cols.append(col)
    
#     print("--- Encoding Summary ---")
#     print(f"Applying Label Encoding to: {label_encode_cols}")
#     print(f"Applying One-Hot Encoding to: {ohe_cols}")
#     print(f"Applying {high_cardinality_strategy} Encoding to: {high_card_cols}")

#     # Apply Label Encoding
#     for col in label_encode_cols:
#         le = LabelEncoder()
#         encoded_df[col] = le.fit_transform(encoded_df[col])
    
#     # Apply One-Hot Encoding
#     encoded_df = pd.get_dummies(encoded_df, columns=ohe_cols, drop_first=True)
    
#     # Apply high-cardinality strategy
#     if high_card_cols:
#         if high_cardinality_strategy == 'binary':
#             encoder = ce.BinaryEncoder(cols=high_card_cols)
#             encoded_df = encoder.fit_transform(encoded_df)
#         elif high_cardinality_strategy == 'frequency':
#             for col in high_card_cols:
#                 freq_map = encoded_df[col].value_counts(normalize=True)
#                 encoded_df[f'{col}_freq'] = encoded_df[col].map(freq_map)
#                 encoded_df = encoded_df.drop(columns=[col])
        
#     return encoded_df

# # --- How to use this function ---
# # Example 1: Use Binary Encoding for high-cardinality columns
# df_encoded_binary = magic_auto_encode(df1, low_cardinality_threshold=5, high_cardinality_threshold=50, high_cardinality_strategy='binary', exclude_cols=['flight'])

# # Example 2: Use Frequency Encoding for high-cardinality columns
# df_encoded_freq = magic_auto_encode(df1, low_cardinality_threshold=5, high_cardinality_threshold=50, high_cardinality_strategy='frequency', exclude_cols=['flight'])

In [ ]:
# import pandas as pd
# import numpy as np
# from sklearn.preprocessing import LabelEncoder
# import category_encoders as ce

# def magic_auto_encode_with_target(df, target_col, low_cardinality_threshold=5, high_cardinality_threshold=50, high_cardinality_strategy='binary', exclude_cols=[]):
#     """
#     Automatically applies different encoders based on cardinality, including
#     Target Encoding for high-cardinality columns.

#     Args:
#         df (pd.DataFrame): The DataFrame to encode.
#         target_col (str): The name of the target column.
#         low_cardinality_threshold (int): Max unique values for Label Encoding.
#         high_cardinality_threshold (int): Min unique values for high-cardinality strategy.
#         high_cardinality_strategy (str): 'binary', 'frequency', or 'target'.
#         exclude_cols (list): A list of columns to exclude from encoding.

#     Returns:
#         pd.DataFrame: A new DataFrame with all categorical columns encoded.
#     """
#     # Check if the target column exists in the DataFrame
#     if high_cardinality_strategy == 'target' and target_col not in df.columns:
#         raise ValueError(f"Target column '{target_col}' not found in DataFrame.")

#     encoded_df = df.copy()
    
#     # Exclude the target column and manually excluded columns from encoding
#     excluded_from_encoding = exclude_cols + [target_col]
#     all_cat_cols = [col for col in encoded_df.select_dtypes(include=['object', 'category']).columns.tolist() 
#                     if col not in excluded_from_encoding]
    
#     label_encode_cols = []
#     ohe_cols = []
#     high_card_cols = []
    
#     for col in all_cat_cols:
#         n_unique = encoded_df[col].nunique()
#         if n_unique <= low_cardinality_threshold:
#             label_encode_cols.append(col)
#         elif n_unique > high_cardinality_threshold:
#             high_card_cols.append(col)
#         else:
#             ohe_cols.append(col)
    
#     print("--- Encoding Summary ---")
#     print(f"Applying Label Encoding to: {label_encode_cols}")
#     print(f"Applying One-Hot Encoding to: {ohe_cols}")
#     print(f"Applying {high_cardinality_strategy} Encoding to: {high_card_cols}")

#     # Apply Label Encoding
#     for col in label_encode_cols:
#         le = LabelEncoder()
#         encoded_df[col] = le.fit_transform(encoded_df[col])
    
#     # Apply One-Hot Encoding
#     encoded_df = pd.get_dummies(encoded_df, columns=ohe_cols, drop_first=True)
    
#     # Apply high-cardinality strategy
#     if high_card_cols:
#         if high_cardinality_strategy == 'binary':
#             encoder = ce.BinaryEncoder(cols=high_card_cols)
#             encoded_df = encoder.fit_transform(encoded_df)
#         elif high_cardinality_strategy == 'frequency':
#             for col in high_card_cols:
#                 freq_map = encoded_df[col].value_counts(normalize=True)
#                 encoded_df[f'{col}_freq'] = encoded_df[col].map(freq_map)
#                 encoded_df = encoded_df.drop(columns=[col])
#         elif high_cardinality_strategy == 'target':
#             # Target Encoding using a cross-validation approach to prevent data leakage
#             encoder = ce.TargetEncoder(cols=high_card_cols)
#             encoded_df = encoder.fit_transform(encoded_df, encoded_df[target_col])
            
#     return encoded_df

# # --- HOW TO USE WITH YOUR DATAFRAME ---
# # Assuming your DataFrame is named 'df' and your target column is 'target_label'
# # Replace 'target_label' with the actual name of your target column.
# # Replace the list ['col1_to_exclude', 'col2_to_exclude'] with your actual columns to exclude.

# # Example 1: Use Target Encoding for high-cardinality columns
# my_encoded_df = magic_auto_encode_with_target(
#     df=df,
#     target_col='target_label',  # <-- REPLACE WITH YOUR TARGET COLUMN NAME
#     low_cardinality_threshold=10, 
#     high_cardinality_threshold=100, 
#     high_cardinality_strategy='target',
#     exclude_cols=['col1_to_exclude', 'col2_to_exclude'] # <-- REPLACE WITH YOUR COLUMNS TO EXCLUDE
# )

# print(my_encoded_df.head())


In [ ]:
df_encoded.head()

In [ ]:
# import pandas as pd
# from sklearn.preprocessing import LabelEncoder

# def auto_encode_df(df, ordinal_cols):
#     """
#     Automatically applies Label Encoding to ordinal columns and One-Hot Encoding
#     to all other categorical columns.
    
#     Args:
#         df (pd.DataFrame): The DataFrame to encode.
#         ordinal_cols (list): A list of column names to be treated as ordinal.
        
#     Returns:
#         pd.DataFrame: A new DataFrame with all categorical columns encoded.
#     """
#     encoded_df = df.copy()
    
#     # Identify all categorical columns
#     all_cat_cols = encoded_df.select_dtypes(include=['object', 'category']).columns.tolist()
    
#     # Identify nominal columns by excluding the ordinal ones
#     nominal_cols = [col for col in all_cat_cols if col not in ordinal_cols]
    
#     # Apply One-Hot Encoding to nominal columns
#     encoded_df = pd.get_dummies(encoded_df, columns=nominal_cols, drop_first=True)
    
#     # Apply Label Encoding to ordinal columns
#     for col in ordinal_cols:
#         le = LabelEncoder()
#         encoded_df[col] = le.fit_transform(encoded_df[col])
        
#     return encoded_df

# # --- How to use this function ---
# # Assuming df1 is your cleaned DataFrame and you know 'stops' is ordinal.
# # You only need to define this one list.
# ordinal_columns_to_encode = ['stops']

# # Call the function to get a new, fully encoded DataFrame.
# df_encoded = auto_encode_df(df1, ordinal_columns_to_encode)

# print("Original DataFrame columns:", df1.columns.tolist())
# print("Encoded DataFrame columns:", df_encoded.columns.tolist())
# print("\nFirst 5 rows of the encoded DataFrame:")
# print(df_encoded.head())

In [ ]:
import pandas as pd
from scipy.stats import shapiro
from pandas.api.types import is_numeric_dtype
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import numpy as np

# This script is designed to work with your DataFrame, df1.
# Please ensure your df1 DataFrame is loaded and available in your environment
# before running this code.

# We will create a copy of df1 to store the scaled data
df1_scaled = df_encoded.copy()

def shapiro_test_column(data_series, column_name, alpha=0.05):
    """
    Performs the Shapiro-Wilk test for normality on a single column.
    Returns True if the data is likely normal, False otherwise.
    Includes error handling for data where the test cannot be performed.
    """
    # Shapiro-Wilk test requires at least 3 samples
    if len(data_series.dropna()) < 3:
        print(f"Skipping '{column_name}': not enough data points for the test.")
        return False
        
    try:
        # Perform the Shapiro-Wilk test
        stat, p_value = shapiro(data_series.dropna())
        
        print(f"\n--- Results for column: '{column_name}' ---")
        print(f"Shapiro-Wilk Test Statistic: {stat:.4f}")
        print(f"P-value: {p_value:.4f}")
        
        if p_value > alpha:
            print("Conclusion: The data is likely from a normal distribution.")
            return True
        else:
            print("Conclusion: The data is likely NOT from a normal distribution.")
            return False
            
    except Exception as e:
        print(f"An error occurred while testing column '{column_name}': {e}")
        return False

# Lists to store the names of columns based on their distribution
normal_columns = []
non_normal_columns = []

print("Starting normality checks on all columns in your DataFrame (df1)...\n")

# Loop through each column in the DataFrame
for column in df1.columns:
    # Check if the column is a numerical type before attempting the test
    if is_numeric_dtype(df1[column]):
        is_normal = shapiro_test_column(df1[column], column)
        
        if is_normal:
            normal_columns.append(column)
        else:
            non_normal_columns.append(column)
            
    else:
        print(f"--- Skipping non-numerical column: '{column}' ---")

print("\n" + "="*50)
print("Normality checks complete.")
print(f"Columns identified as 'normal' (p-value > 0.05): {normal_columns}")
print(f"Columns identified as 'not normal' (p-value <= 0.05): {non_normal_columns}")
print("="*50)

# Apply the appropriate scaling technique to the identified columns
if normal_columns:
    print("\nApplying StandardScaler to normal columns...")
    scaler_std = StandardScaler()
    df1_scaled[normal_columns] = scaler_std.fit_transform(df1[normal_columns])
    print("StandardScaler applied successfully.")

if non_normal_columns:
    print("\nApplying MinMaxScaler to non-normal columns...")
    scaler_minmax = MinMaxScaler()
    df1_scaled[non_normal_columns] = scaler_minmax.fit_transform(df1[non_normal_columns])
    print("MinMaxScaler applied successfully.")

print("\nFirst 5 rows of the scaled DataFrame (df1_scaled):")
print(df1_scaled.head())


In [ ]:
df1_scaled.head()

In [ ]:
# import pandas as pd
# from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import LabelEncoder
# from sklearn.feature_selection import SelectKBest, f_classif, chi2, f_regression
# from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
# from sklearn.feature_selection import SelectFromModel
# import numpy as np

# # This script is designed to run on a DataFrame that has already been
# # preprocessed, including scaling of numerical features and one-hot
# # encoding of categorical features.

# def perform_automated_feature_selection(df, target_column_name, k_features=10):
#     """
#     A versatile function to automatically select features for both classification
#     and regression problems.

#     Args:
#         df (pd.DataFrame): The input DataFrame.
#         target_column_name (str): The name of the target variable column.
#         k_features (int): The number of top features to select using filter methods.

#     Returns:
#         tuple: A tuple containing the new DataFrame with selected features and
#                a list of the names of the selected features.
#     """
#     # --- Step 1: Prepare the Data ---
#     print("Step 1: Preparing data for feature selection...")
#     X = df.drop(target_column_name, axis=1)
#     y = df[target_column_name]

#     # Check if the problem is classification or regression
#     # Based on the user's input, the target is 'duration', which is numerical.
#     # Therefore, this is a regression problem.
#     problem_type = 'regression'
#     print("Detected problem type: Regression (Target variable is numerical)")
    
#     # The target variable 'duration' is already numerical, so no encoding is needed.
#     y_encoded = y
    
#     # Identify numerical and categorical columns from the features
#     numerical_cols = X.select_dtypes(include=np.number).columns
#     categorical_cols = X.select_dtypes(include=['object', 'category']).columns

#     # Handle the case where no categorical columns are found
#     if not categorical_cols.empty:
#         # One-hot encode categorical features
#         X_encoded_categorical = pd.get_dummies(X[categorical_cols], drop_first=True)
#         # Combine numerical and encoded categorical features
#         X_combined = pd.concat([X[numerical_cols], X_encoded_categorical], axis=1)
#     else:
#         print("No categorical columns found. Using only numerical features.")
#         X_combined = X[numerical_cols]

#     print(f"Original features: {len(X.columns)}")
#     print(f"Features after one-hot encoding: {len(X_combined.columns)}")
#     print("-" * 50)

#     # --- Step 2: Apply Filter Methods ---
#     print("Step 2: Applying filter methods...")
    
#     # Use f_regression for numerical features vs. a numerical target
#     selector_filter = SelectKBest(score_func=f_regression, k=k_features)
#     selector_filter.fit(X_combined, y_encoded)
    
#     anova_scores = pd.Series(selector_filter.scores_, index=X_combined.columns)
#     filtered_features = anova_scores.nlargest(k_features).index.tolist()

#     # Note: Chi-Square is not applicable for a regression problem, so we skip it.
#     print(f"Top {k_features} features from filter methods (f_regression): {filtered_features}")
#     print("-" * 50)

#     # --- Step 3: Apply Embedded Method ---
#     print("Step 3: Applying embedded method...")
#     # Use RandomForestRegressor for a regression problem
#     model = RandomForestRegressor(n_estimators=100, random_state=42)

#     selector_embedded = SelectFromModel(model, threshold='median', prefit=False)
#     selector_embedded.fit(X_combined, y_encoded)
#     embedded_features = X_combined.columns[selector_embedded.get_support()].tolist()
#     print(f"Features selected by Random Forest: {embedded_features}")
#     print("-" * 50)

#     # --- Step 4: Combine Results and Finalize ---
#     print("Step 4: Combining results and finalizing feature list...")
#     final_selected_features = list(set(filtered_features + embedded_features))
#     df_final = X_combined[final_selected_features]
    
#     print(f"Final selected features ({len(final_selected_features)}): {final_selected_features}")
#     print(f"Final DataFrame shape: {df_final.shape}")
    
#     return df_final, final_selected_features

# # --- Execution ---
# # Call the function to perform feature selection with 'duration' as the target
# # This assumes 'df1_scaled' is already defined in your environment.
# df_final_features, selected_features_list = perform_automated_feature_selection(df1_scaled, 'duration', k_features=10)

# # Display the final DataFrame and list of features
# print("\n--- Final Results ---")
# print("Selected Features List:")
# print(selected_features_list)
# print("\nFinal DataFrame with selected features (first 5 rows):")
# print(df_final_features.head())


In [ ]:
#magic function to apply feature selction with chi, embedded(reg and classi), filter for reg and classi, 

import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_selection import SelectKBest, f_classif, chi2, f_regression
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.feature_selection import SelectFromModel
import numpy as np

def perform_automated_feature_selection(df, target_column_name, k_features=10):
    """
    Automatically selects the most important features for a dataset.
    This function intelligently switches between methods for classification
    and regression problems.

    Args:
        df (pd.DataFrame): The DataFrame to analyze. It should already be
                           preprocessed, with numerical columns scaled and
                           categorical columns one-hot encoded.
        target_column_name (str): The name of the column you want to predict.
        k_features (int): The number of features to select using statistical tests.

    Returns:
        tuple: A new DataFrame with only the selected features, and a list of
               the names of those features.
    """
    # --- Step 1: Prepare the Data (Separate features from target) ---
    print("Step 1: Separating features (X) and target (y)...")
    X = df.drop(target_column_name, axis=1)
    y = df[target_column_name]

    # Automatically decide if it's a classification or regression problem
    # If the target has few unique values or is an object type, it's classification.
    if y.dtype == 'object' or y.dtype == 'category' or y.nunique() <= 20:
        problem_type = 'classification'
        print(f"Problem Type: Classification (Target '{target_column_name}' has {y.nunique()} unique values)")
        # Encode the target variable for classification models
        le = LabelEncoder()
        y_encoded = le.fit_transform(y)
    else:
        problem_type = 'regression'
        print(f"Problem Type: Regression (Target '{target_column_name}' is numerical)")
        y_encoded = y
    
    # Identify numerical and categorical columns from the features
    numerical_cols = X.select_dtypes(include=np.number).columns
    categorical_cols = X.select_dtypes(include=['object', 'category']).columns

    # --- Step 2: Handle Categorical Features and Combine Data ---
    print("Step 2: Handling categorical features and combining data...")
    if not categorical_cols.empty:
        # One-hot encode categorical features if they exist
        X_encoded_categorical = pd.get_dummies(X[categorical_cols], drop_first=True)
        X_combined = pd.concat([X[numerical_cols], X_encoded_categorical], axis=1)
    else:
        print("No categorical columns found. Using only numerical features.")
        X_combined = X[numerical_cols]

    print(f"Features ready for selection. Total columns: {len(X_combined.columns)}")
    print("-" * 50)
    
    # --- Step 3: Choose Methods Based on Problem Type ---
    print("Step 3: Choosing feature selection methods...")
    if problem_type == 'classification':
        score_func = f_classif
        model = RandomForestClassifier(n_estimators=100, random_state=42)
    else: # Regression
        score_func = f_regression
        model = RandomForestRegressor(n_estimators=100, random_state=42)

    # --- Step 4: Apply Filter Methods (Statistical Tests) ---
    print("Step 4: Applying statistical filter methods...")
    filtered_features = []
    
    if problem_type == 'classification' and not categorical_cols.empty:
        # Use Chi-Square for categorical features in classification
        selector_chi2 = SelectKBest(score_func=chi2, k='all')
        selector_chi2.fit(X_combined[X_encoded_categorical.columns], y_encoded)
        chi2_scores = pd.Series(selector_chi2.scores_, index=X_encoded_categorical.columns)
        
        # Use f_classif for numerical features
        selector_anova = SelectKBest(score_func=score_func, k='all')
        selector_anova.fit(X_combined[numerical_cols], y_encoded)
        anova_scores = pd.Series(selector_anova.scores_, index=numerical_cols)
        
        # Concatenate scores and get the top K features
        combined_scores = pd.concat([anova_scores, chi2_scores]).sort_values(ascending=False)
        filtered_features = combined_scores.head(k_features).index.tolist()
        print(f"Top {k_features} features from f_classif and chi-square: {filtered_features}")
        
    else: # This applies to all regression problems and classification without categorical data
        selector_filter = SelectKBest(score_func=score_func, k=k_features)
        selector_filter.fit(X_combined, y_encoded)
        filtered_scores = pd.Series(selector_filter.scores_, index=X_combined.columns)
        filtered_features = filtered_scores.nlargest(k_features).index.tolist()
        print(f"Top {k_features} features from {score_func.__name__}: {filtered_features}")

    print("-" * 50)

    # --- Step 5: Apply Embedded Method (Random Forest) ---
    print("Step 5: Applying embedded method (Random Forest)...")
    selector_embedded = SelectFromModel(model, threshold='median', prefit=False)
    selector_embedded.fit(X_combined, y_encoded)
    embedded_features = X_combined.columns[selector_embedded.get_support()].tolist()
    print(f"Features selected by Random Forest: {embedded_features}")
    print("-" * 50)

    # --- Step 6: Combine Results and Finalize ---
    print("Step 6: Combining results and creating final DataFrame...")
    final_selected_features = list(set(filtered_features + embedded_features))
    df_final = X_combined[final_selected_features]
    
    print(f"Final selected features ({len(final_selected_features)}): {final_selected_features}")
    print(f"Final DataFrame shape: {df_final.shape}")
    
    return df_final, final_selected_features

# --- Execution ---
# Call the function with your DataFrame and target column name.
# Replace 'df1_scaled' and 'duration' with your actual variables.
df_final_features, selected_features_list = perform_automated_feature_selection(df1_scaled, 'duration', k_features=10)

# Display the final DataFrame and list of features
print("\n--- Final Results ---")
print("Selected Features List:")
print(selected_features_list)
print("\nFinal DataFrame with selected features (first 5 rows):")
print(df_final_features.head())


In [ ]:
print(df_final_features.head())

In [ ]:
# import pandas as pd
# import numpy as np
# from sklearn.decomposition import PCA
# from sklearn.preprocessing import StandardScaler
# from pandas.api.types import is_numeric_dtype
# from scipy.stats import shapiro

# # --- PART 1: SIMULATE YOUR DATAFRAME AND SCALING ---
# # This simulates the steps in your workflow that lead to df1_scaled.
# # The key here is to correctly unpack the result.

# # 1. Create a sample DataFrame to simulate your data.
# np.random.seed(42)
# df_original = pd.DataFrame({
#     'normal_col': np.random.normal(loc=10, scale=2, size=100),
#     'non_normal_col': np.random.exponential(scale=3, size=100),
#     'categorical_col': np.random.choice(['A', 'B', 'C'], size=100),
#     'target_col': np.random.randint(0, 2, size=100)
# })

# # 2. A function that performs scaling and returns multiple items (a DataFrame and scalers).
# def apply_scaling_by_normality(df_input, exclude_cols=[]):
#     """
#     Simulates a scaling function that returns a scaled DataFrame and a scaler model.
#     """
#     df_output = df_input.copy()
#     scalers = {}

#     normal_columns = [col for col in df_input.select_dtypes(include=np.number).columns
#                       if col not in exclude_cols and shapiro(df_input[col].dropna())[1] > 0.05]
#     non_normal_columns = [col for col in df_input.select_dtypes(include=np.number).columns
#                           if col not in exclude_cols and shapiro(df_input[col].dropna())[1] <= 0.05]

#     if normal_columns:
#         scaler_std = StandardScaler()
#         df_output[normal_columns] = scaler_std.fit_transform(df_input[normal_columns])
#         scalers['standard'] = scaler_std
#     else:
#         # A simple check to ensure a scaler is still available.
#         # This will only be run if there are no 'normal' columns.
#         scaler_std = StandardScaler()
#         scalers['standard'] = scaler_std
    
#     if non_normal_columns:
#         scaler_minmax = MinMaxScaler()
#         df_output[non_normal_columns] = scaler_minmax.fit_transform(df_input[non_normal_columns])
#         scalers['minmax'] = scaler_minmax
#     else:
#         # A simple check to ensure a scaler is still available.
#         scaler_minmax = MinMaxScaler()
#         scalers['minmax'] = scaler_minmax


#     # The return value is a tuple of two items.
#     return df_output, scalers

# # 3. The CORRECT way to call the scaling function.
# # We use two variables to "unpack" the two items returned by the function.
# # This ensures that df1_scaled is a DataFrame, not a tuple.
# df1_scaled, scaler_models = apply_scaling_by_normality(df_original, exclude_cols=['target_col'])
# print("Successfully unpacked scaling function output.")
# print(f"Type of df1_scaled: {type(df1_scaled)}")
# print("="*50)


# # --- PART 2: THE PCA FUNCTION WITH AN EXPLICIT ERROR CHECK ---

# def apply_pca(df, n_components=0.95):
#     """
#     Apply PCA retaining n_components variance (float) or number of components (int).
#     """
#     # --- CRITICAL FIX: Explicitly check if the input is a DataFrame ---
#     if not isinstance(df, pd.DataFrame):
#         print(f"ERROR: Expected a pandas DataFrame, but received a '{type(df).__name__}' object.")
#         print("This means a previous function call was not unpacked correctly.")
#         print("Please ensure the scaling function output is assigned to two variables, e.g., 'df_scaled, scaler_model = scale_function(...)'.")
#         # Returning here to prevent the code from crashing.
#         return None, None
    
#     # This line will now work because 'df' is a pandas DataFrame.
#     df_numerical = df.select_dtypes(include=np.number)
    
#     # We'll drop any categorical or non-numerical columns here to be safe
#     df_numerical = df.select_dtypes(include=np.number).drop(columns=['target_col'], errors='ignore')
    
#     if df_numerical.shape[1] < 2:
#         print("Not enough numerical columns for PCA.")
#         return df_numerical, None

#     pca = PCA(n_components=n_components)
#     components = pca.fit_transform(df_numerical)
    
#     col_names = [f'PC{i+1}' for i in range(components.shape[1])]
#     df_pca = pd.DataFrame(components, columns=col_names, index=df_numerical.index)
    
#     print(f"Explained variance: {pca.explained_variance_ratio_.sum():.4f}")
#     print(f"Number of components selected: {pca.n_components_}")
#     print(f"Original shape: {df_numerical.shape}")
#     print(f"PCA-transformed shape: {df_pca.shape}")
    
#     return df_pca, pca

# # --- CORRECTED CALL: This line will now execute without the TypeError ---
# # We are passing the correctly unpacked DataFrame to the function.
# df1_dimensionality, pca_model = apply_pca(df1_scaled, n_components=0.95)

# if df1_dimensionality is not None:
#     print("\nPCA applied successfully. The new DataFrame is ready.")
#     print(df1_dimensionality.head())



In [ ]:
df_final_features.head()

In [ ]:
df_final_features.shape

In [ ]:
# Assuming 'df1_scaled' is your full, preprocessed DataFrame with the 'duration' column.
# The `perform_automated_feature_selection` function returns a DataFrame with only the selected features.
df_final_features, selected_features_list = perform_automated_feature_selection(df1_scaled, 'duration', k_features=10)

# --- Define X and y for the train-test split ---
# X will be the DataFrame of selected features returned from your function.
X = df_final_features

# y will be the original target column from your starting DataFrame.
y = df1_scaled['duration']

# --- Perform the train-test split ---
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2,
    random_state=42
)

print(f"Training features (X_train) shape: {X_train.shape}")
print(f"Testing features (X_test) shape: {X_test.shape}")

In [ ]:
# import pandas as pd
# import numpy as np
# from sklearn.metrics import mean_absolute_error, r2_score

# # Regression Models
# from sklearn.linear_model import LinearRegression, RidgeCV, LassoCV, ElasticNetCV
# from sklearn.tree import DecisionTreeRegressor
# from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, AdaBoostRegressor
# from sklearn.svm import SVR
# from sklearn.neighbors import KNeighborsRegressor
# from sklearn.preprocessing import PolynomialFeatures
# from sklearn.pipeline import Pipeline
# from sklearn.neural_network import MLPRegressor
# from xgboost import XGBRegressor
# from lightgbm import LGBMRegressor
# from catboost import CatBoostRegressor

# # ===============================================================================
# # This script assumes you have already defined and split your data into
# # X_train, X_test, y_train, and y_test.
# # You can now proceed directly with model training and evaluation.
# # ===============================================================================

# # Define a dictionary of all the regression models to test
# models = {
#     "Linear Regression": LinearRegression(),
#     "Ridge Regression": RidgeCV(),
#     "Lasso Regression": LassoCV(),
#     "ElasticNet Regression": ElasticNetCV(),
#     "Decision Tree": DecisionTreeRegressor(random_state=42),
#     "Random Forest": RandomForestRegressor(random_state=42),
#     "Gradient Boosting": GradientBoostingRegressor(random_state=42),
#     "AdaBoost": AdaBoostRegressor(random_state=42),
#     "XGBoost": XGBRegressor(random_state=42),
#     "LightGBM": LGBMRegressor(random_state=42),
#     "CatBoost": CatBoostRegressor(random_state=42, verbose=0),
#     "SVR": SVR(),
#     "K-Nearest Neighbors": KNeighborsRegressor(),
#     "MLP Regressor": MLPRegressor(random_state=42, max_iter=500),
#     # Using PolynomialFeatures requires a pipeline
#     "Polynomial Regression": Pipeline([
#         ('poly', PolynomialFeatures(degree=2)),
#         ('linreg', LinearRegression())
#     ])
# }

# # Loop through each model, train it, and evaluate its performance
# print("\n--- Starting Model Training and Evaluation ---")
# for name, model in models.items():
#     print(f"\nTraining and evaluating: {name}...")
#     try:
#         # Train the model
#         model.fit(X_train, y_train)

#         # Make predictions on the test set
#         y_pred = model.predict(X_test)

#         # Calculate and print performance metrics
#         mae = mean_absolute_error(y_test, y_pred)
#         r2 = r2_score(y_test, y_pred)

#         print(f"  Mean Absolute Error (MAE): {mae:.4f}")
#         print(f"  R-squared (R²): {r2:.4f}")

#     except Exception as e:
#         print(f"  Error with {name}: {e}")
#     finally:
#         print("-" * 50)

# print("All models have been evaluated.")

In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_absolute_error, r2_score

# Import necessary models for the first set
from sklearn.linear_model import LinearRegression, RidgeCV, LassoCV, ElasticNetCV
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, AdaBoostRegressor

# ===============================================================================
# Run this code in a separate cell.
# It assumes X_train, X_test, y_train, and y_test are already defined.
# ===============================================================================

# Define a dictionary of the first set of regression models to test
models_set1 = {
    "Linear Regression": LinearRegression(),
    "Ridge Regression": RidgeCV(),
    "Lasso Regression": LassoCV(),
    "ElasticNet Regression": ElasticNetCV(),
    "Decision Tree": DecisionTreeRegressor(random_state=42),
    "Random Forest": RandomForestRegressor(random_state=42),
    "Gradient Boosting": GradientBoostingRegressor(random_state=42),
    "AdaBoost": AdaBoostRegressor(random_state=42),
}

# Loop through each model, train it, and evaluate its performance
print("\n--- Starting Model Training and Evaluation (Set 1) ---")
for name, model in models_set1.items():
    print(f"\nTraining and evaluating: {name}...")
    try:
        # Train the model
        model.fit(X_train, y_train)

        # Make predictions on the test set
        y_pred = model.predict(X_test)

        # Calculate and print performance metrics
        mae = mean_absolute_error(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)

        mape = np.mean(np.abs((y_test - y_pred) / y_test.replace(0, 1e-10))) * 100

        print(f"  Mean Absolute Error (MAE): {mae:.4f}")
        print(f"  R-squared (R²): {r2:.4f}")
        print(f"  Mean Absolute Percentage Error (MAPE): {mape:.2f}%")

    

    except Exception as e:
        print(f"  Error with {name}: {e}")
    finally:
        print("-" * 50)

print("Set 1 models have been evaluated.")


In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_absolute_error, r2_score

# Import necessary models for the second set
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline
from sklearn.neural_network import MLPRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor

# ===============================================================================
# Run this code in a separate cell.
# It assumes X_train, X_test, y_train, and y_test are already defined.
# ===============================================================================

# Define a dictionary of the second set of regression models to test
models_set2 = {
    "XGBoost": XGBRegressor(random_state=42),
    "LightGBM": LGBMRegressor(random_state=42),
    "CatBoost": CatBoostRegressor(random_state=42, verbose=0),
    # "SVR": SVR(),
    "K-Nearest Neighbors": KNeighborsRegressor(),
    "MLP Regressor": MLPRegressor(random_state=42, max_iter=500),
    # Using PolynomialFeatures requires a pipeline
    "Polynomial Regression": Pipeline([
        ('poly', PolynomialFeatures(degree=2)),
        ('linreg', LinearRegression())
    ])
}

# Loop through each model, train it, and evaluate its performance
print("\n--- Starting Model Training and Evaluation (Set 2) ---")
for name, model in models_set2.items():
    print(f"\nTraining and evaluating: {name}...")
    try:
        # Train the model
        model.fit(X_train, y_train)

        # Make predictions on the test set
        y_pred = model.predict(X_test)

        # Calculate and print performance metrics
        mae = mean_absolute_error(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)

        mape = np.mean(np.abs((y_test - y_pred) / y_test.replace(0, 1e-10))) * 100

        print(f"  Mean Absolute Error (MAE): {mae:.4f}")
        print(f"  R-squared (R²): {r2:.4f}")
        print(f"  Mean Absolute Percentage Error (MAPE): {mape:.2f}%")

    except Exception as e:
        print(f"  Error with {name}: {e}")
    finally:
        print("-" * 50)

print("Set 2 models have been evaluated.")


In [ ]:
# Import all necessary libraries
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, cross_validate, KFold, GridSearchCV, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression, RidgeCV, LassoCV, ElasticNetCV
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, AdaBoostRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neural_network import MLPRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Assume df_final_features and df1_scaled['duration'] are already defined
# X = df_final_features
# y = df1_scaled['duration']

# --- Step 1: Split data into training and testing sets ---
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# --- Step 2: Create a smaller training sample for SVR from the existing training data ---
# SVR can be slow on large datasets. This creates a 10% subsample of the
# training data specifically for the SVR model.
X_train_svr, _, y_train_svr, _ = train_test_split(X_train, y_train, test_size=0.9, random_state=42)
print(f"Created a smaller training sample for SVR with {len(X_train_svr)} observations.")

# --- Step 3: Define Preprocessing ---
# Identify numeric and categorical features
numeric_features = X.select_dtypes(include=np.number).columns
categorical_features = X.select_dtypes(include='object').columns

# Create preprocessing steps for numeric and categorical data
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Use ColumnTransformer to apply the transformations to the correct columns
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])


# --- Step 4: Define custom evaluation functions ---
def mape(y_true, y_pred):
    """
    Calculates the Mean Absolute Percentage Error (MAPE).
    """
    # Avoid division by zero by adding a small epsilon
    epsilon = 1e-10
    return np.mean(np.abs((y_true - y_pred) / (y_true + epsilon))) * 100

def rmsle(y_true, y_pred):
    """
    Calculates the Root Mean Squared Logarithmic Error (RMSLE).
    """
    # Make sure predictions are non-negative and add 1 for log
    y_pred[y_pred < 0] = 0
    return np.sqrt(mean_squared_error(np.log1p(y_true), np.log1p(y_pred)))

def adj_r2(y_true, y_pred, n_features):
    """
    Calculates the Adjusted R-squared.
    n is the number of observations, p is the number of predictors.
    """
    n = y_true.shape[0]
    p = n_features
    r2 = r2_score(y_true, y_pred)
    if p >= n - 1: # Avoid division by zero if p is too large
        return np.nan
    return 1 - (1 - r2) * (n - 1) / (n - p - 1)


# --- Step 5: Hyperparameter Tuning ---
print("\n--- Performing Hyperparameter Tuning for selected models ---")

# --- a) Decision Tree Regressor (Grid Search) ---
print("\n  > Starting Grid Search for Decision Tree...")
dt_pipeline = Pipeline(steps=[('preprocessor', preprocessor), ('regressor', DecisionTreeRegressor(random_state=42))])
param_grid_dt = {
    'regressor__max_depth': [5, 10, 15],
    'regressor__min_samples_leaf': [1, 5, 10],
    'regressor__criterion': ['squared_error', 'absolute_error']
}
grid_search_dt = GridSearchCV(dt_pipeline, param_grid_dt, cv=5, scoring='r2', n_jobs=-1, verbose=1)
grid_search_dt.fit(X_train, y_train)
best_dt_model = grid_search_dt.best_estimator_
print(f"  > Best Decision Tree parameters: {grid_search_dt.best_params_}")


# --- b) SVR (Grid Search) ---
print("\n  > Starting Grid Search for SVR...")
svr_pipeline = Pipeline(steps=[('preprocessor', preprocessor), ('regressor', SVR())])
param_grid_svr = {
    'regressor__kernel': ['linear', 'rbf'],
    'regressor__C': [0.1, 1, 10],
    'regressor__epsilon': [0.1, 0.2]
}
grid_search_svr = GridSearchCV(svr_pipeline, param_grid_svr, cv=5, scoring='r2', n_jobs=-1, verbose=1)
# Use the smaller subsample for SVR tuning
grid_search_svr.fit(X_train_svr, y_train_svr)
best_svr_model = grid_search_svr.best_estimator_
print(f"  > Best SVR parameters: {grid_search_svr.best_params_}")


# --- c) Random Forest Regressor (Randomized Search) ---
print("\n  > Starting Randomized Search for Random Forest...")
rf_pipeline = Pipeline(steps=[('preprocessor', preprocessor), ('regressor', RandomForestRegressor(random_state=42))])
param_distributions_rf = {
    'regressor__n_estimators': [100, 200, 300],
    'regressor__max_depth': [10, 20, None],
    'regressor__min_samples_leaf': [1, 2, 4],
    'regressor__min_samples_split': [2, 5, 10]
}
random_search_rf = RandomizedSearchCV(rf_pipeline, param_distributions_rf, n_iter=10, cv=3, scoring='r2', n_jobs=-1, verbose=1, random_state=42)
random_search_rf.fit(X_train, y_train)
best_rf_model = random_search_rf.best_estimator_
print(f"  > Best Random Forest parameters: {random_search_rf.best_params_}")


# --- d) Gradient Boosting Regressor (Randomized Search) ---
print("\n  > Starting Randomized Search for Gradient Boosting...")
gb_pipeline = Pipeline(steps=[('preprocessor', preprocessor), ('regressor', GradientBoostingRegressor(random_state=42))])
param_distributions_gb = {
    'regressor__n_estimators': [100, 200, 300],
    'regressor__learning_rate': [0.01, 0.1, 0.2],
    'regressor__max_depth': [3, 5, 8],
    'regressor__subsample': [0.8, 0.9, 1.0]
}
random_search_gb = RandomizedSearchCV(gb_pipeline, param_distributions_gb, n_iter=10, cv=3, scoring='r2', n_jobs=-1, verbose=1, random_state=42)
random_search_gb.fit(X_train, y_train)
best_gb_model = random_search_gb.best_estimator_
print(f"  > Best Gradient Boosting parameters: {random_search_gb.best_params_}")


# --- e) XGBoost Regressor (Randomized Search) ---
print("\n  > Starting Randomized Search for XGBoost...")
xgb_pipeline = Pipeline(steps=[('preprocessor', preprocessor), ('regressor', XGBRegressor(random_state=42, objective='reg:squarederror'))])
param_distributions_xgb = {
    'regressor__n_estimators': [100, 200, 300],
    'regressor__learning_rate': [0.01, 0.1, 0.2],
    'regressor__max_depth': [3, 5, 8],
    'regressor__subsample': [0.8, 0.9, 1.0]
}
random_search_xgb = RandomizedSearchCV(xgb_pipeline, param_distributions_xgb, n_iter=10, cv=3, scoring='r2', n_jobs=-1, verbose=1, random_state=42)
random_search_xgb.fit(X_train, y_train)
best_xgb_model = random_search_xgb.best_estimator_
print(f"  > Best XGBoost parameters: {random_search_xgb.best_params_}")


# --- Step 6: Create a dictionary of all regression models (tuned and untuned) ---
# This dictionary now includes all our best-performing, tuned models for a
# comprehensive comparison against the baseline models.
models = {
    'Linear Regression': Pipeline(steps=[('preprocessor', preprocessor), ('regressor', LinearRegression())]),
    'RidgeCV': Pipeline(steps=[('preprocessor', preprocessor), ('regressor', RidgeCV(alphas=np.logspace(-6, 6, 13)))]),
    'LassoCV': Pipeline(steps=[('preprocessor', preprocessor), ('regressor', LassoCV(alphas=np.logspace(-6, 6, 13), cv=5))]),
    'ElasticNetCV': Pipeline(steps=[('preprocessor', preprocessor), ('regressor', ElasticNetCV(alphas=np.logspace(-6, 6, 13), cv=5))]),
    'Decision Tree Regressor': Pipeline(steps=[('preprocessor', preprocessor), ('regressor', DecisionTreeRegressor(random_state=42))]),
    'Tuned Decision Tree Regressor': best_dt_model,
    'Random Forest Regressor': Pipeline(steps=[('preprocessor', preprocessor), ('regressor', RandomForestRegressor(random_state=42))]),
    'Tuned Random Forest Regressor': best_rf_model,
    'Gradient Boosting Regressor': Pipeline(steps=[('preprocessor', preprocessor), ('regressor', GradientBoostingRegressor(random_state=42))]),
    'Tuned Gradient Boosting Regressor': best_gb_model,
    'AdaBoost Regressor': Pipeline(steps=[('preprocessor', preprocessor), ('regressor', AdaBoostRegressor(random_state=42))]),
    'SVR': Pipeline(steps=[('preprocessor', preprocessor), ('regressor', SVR())]),
    'Tuned SVR': best_svr_model,
    'KNeighbors Regressor': Pipeline(steps=[('preprocessor', preprocessor), ('regressor', KNeighborsRegressor(n_neighbors=5))]),
    'MLP Regressor': Pipeline(steps=[('preprocessor', preprocessor), ('regressor', MLPRegressor(hidden_layer_sizes=(100,), max_iter=500, random_state=42))]),
    'XGB Regressor': Pipeline(steps=[('preprocessor', preprocessor), ('regressor', XGBRegressor(objective='reg:squarederror', random_state=42))]),
    'Tuned XGB Regressor': best_xgb_model,
    'LGBM Regressor': Pipeline(steps=[('preprocessor', preprocessor), ('regressor', LGBMRegressor(random_state=42))]),
    'CatBoost Regressor': Pipeline(steps=[('preprocessor', preprocessor), ('regressor', CatBoostRegressor(verbose=0, random_state=42))])
}

# --- Step 7: Train and Evaluate all models in a loop ---
print("\n--- Training and Evaluating all Regression Models ---")

# First, get the number of features after preprocessing to calculate Adjusted R2
X_train_transformed = preprocessor.fit_transform(X_train)
n_features = X_train_transformed.shape[1]

for name, model in models.items():
    print(f"\nTraining {name}...")
    try:
        if 'SVR' in name:
            # Use the subsampled data for SVR
            model.fit(X_train_svr, y_train_svr)
            predictions = model.predict(X_test)
        else:
            # Use the full training data for all other models
            model.fit(X_train, y_train)
            predictions = model.predict(X_test)
        
        # Calculate all requested metrics
        mse = mean_squared_error(y_test, predictions)
        mae = mean_absolute_error(y_test, predictions)
        rmse = np.sqrt(mse)
        rmsle_score = rmsle(y_test, predictions)
        r2 = r2_score(y_test, predictions)
        adj_r2_score = adj_r2(y_test, predictions, n_features)
        mape_score = mape(y_test, predictions)
        
        print(f"--- {name} Results ---")
        print(f"Mean Absolute Error (MAE): {mae:.2f}")
        print(f"Mean Squared Error (MSE): {mse:.2f}")
        print(f"Root Mean Squared Error (RMSE): {rmse:.2f}")
        print(f"Root Mean Squared Logarithmic Error (RMSLE): {rmsle_score:.4f}")
        print(f"R-squared (R²): {r2:.4f}")
        print(f"Adjusted R-squared (Adj_R²): {adj_r2_score:.4f}")
        print(f"Mean Absolute Percentage Error (MAPE): {mape_score:.2f}%")
        
    except Exception as e:
        print(f"An error occurred while training {name}: {e}")

print("\n--- All models evaluated. ---")


In [ ]:
# # Import all necessary libraries
# import numpy as np
# import pandas as pd
# from sklearn.model_selection import train_test_split, GridSearchCV
# from sklearn.preprocessing import StandardScaler, OneHotEncoder
# from sklearn.compose import ColumnTransformer
# from sklearn.pipeline import Pipeline
# from sklearn.impute import SimpleImputer
# from sklearn.linear_model import LinearRegression, RidgeCV, LassoCV, ElasticNetCV
# from sklearn.tree import DecisionTreeRegressor
# from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, AdaBoostRegressor
# from sklearn.svm import SVR
# from sklearn.neighbors import KNeighborsRegressor
# from sklearn.neural_network import MLPRegressor
# from xgboost import XGBRegressor
# from lightgbm import LGBMRegressor
# from catboost import CatBoostRegressor
# from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# # Assume df_final_features and df1_scaled['duration'] are already defined
# # X = df_final_features
# # y = df1_scaled['duration']

# # --- Step 1: Split data into training and testing sets ---
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# # --- Step 2: Create a smaller training sample for SVR ---
# X_train_svr, _, y_train_svr, _ = train_test_split(X_train, y_train, test_size=0.9, random_state=42)
# print(f"Created a smaller training sample for SVR with {len(X_train_svr)} observations.")

# # --- Step 3: Define Preprocessing ---
# numeric_features = X.select_dtypes(include=np.number).columns
# categorical_features = X.select_dtypes(include='object').columns

# numeric_transformer = Pipeline(steps=[
#     ('imputer', SimpleImputer(strategy='mean')),
#     ('scaler', StandardScaler())
# ])

# categorical_transformer = Pipeline(steps=[
#     ('imputer', SimpleImputer(strategy='most_frequent')),
#     ('onehot', OneHotEncoder(handle_unknown='ignore'))
# ])

# preprocessor = ColumnTransformer(
#     transformers=[
#         ('num', numeric_transformer, numeric_features),
#         ('cat', categorical_transformer, categorical_features)
#     ])


# # --- Step 4: Define custom evaluation functions ---
# def mape(y_true, y_pred):
#     """Calculates the Mean Absolute Percentage Error (MAPE)."""
#     epsilon = 1e-10
#     return np.mean(np.abs((y_true - y_pred) / (y_true + epsilon))) * 100

# def rmsle(y_true, y_pred):
#     """Calculates the Root Mean Squared Logarithmic Error (RMSLE)."""
#     y_pred[y_pred < 0] = 0
#     return np.sqrt(mean_squared_error(np.log1p(y_true), np.log1p(y_pred)))

# def adj_r2(y_true, y_pred, n_features):
#     """Calculates the Adjusted R-squared."""
#     n = y_true.shape[0]
#     p = n_features
#     r2 = r2_score(y_true, y_pred)
#     if p >= n - 1:
#         return np.nan
#     return 1 - (1 - r2) * (n - 1) / (n - p - 1)


# # --- Step 5: Hyperparameter Tuning (Deep Dive) ---
# print("\n--- Performing Deep Hyperparameter Tuning for top models ---")

# # --- a) Random Forest Regressor (Grid Search) ---
# print("\n  > Starting Grid Search for Random Forest...")
# rf_pipeline = Pipeline(steps=[('preprocessor', preprocessor), ('regressor', RandomForestRegressor(random_state=42))])
# # Focused and narrowed parameter grid for a more exhaustive search
# param_grid_rf = {
#     'regressor__n_estimators': [200, 300],
#     'regressor__max_depth': [20, 25],
#     'regressor__min_samples_leaf': [2, 3]
# }
# grid_search_rf = GridSearchCV(rf_pipeline, param_grid_rf, cv=3, scoring='r2', n_jobs=-1, verbose=1)
# grid_search_rf.fit(X_train, y_train)
# best_rf_model = grid_search_rf.best_estimator_
# print(f"  > Best Random Forest parameters: {grid_search_rf.best_params_}")


# # --- b) Gradient Boosting Regressor (Grid Search) ---
# print("\n  > Starting Grid Search for Gradient Boosting...")
# gb_pipeline = Pipeline(steps=[('preprocessor', preprocessor), ('regressor', GradientBoostingRegressor(random_state=42))])
# # Focused and narrowed parameter grid for a more exhaustive search
# param_grid_gb = {
#     'regressor__n_estimators': [200, 300],
#     'regressor__learning_rate': [0.05, 0.1],
#     'regressor__max_depth': [5, 7],
# }
# grid_search_gb = GridSearchCV(gb_pipeline, param_grid_gb, cv=3, scoring='r2', n_jobs=-1, verbose=1)
# grid_search_gb.fit(X_train, y_train)
# best_gb_model = grid_search_gb.best_estimator_
# print(f"  > Best Gradient Boosting parameters: {grid_search_gb.best_params_}")


# # --- c) XGBoost Regressor (Grid Search) ---
# print("\n  > Starting Grid Search for XGBoost...")
# xgb_pipeline = Pipeline(steps=[('preprocessor', preprocessor), ('regressor', XGBRegressor(random_state=42, objective='reg:squarederror'))])
# # Focused and narrowed parameter grid for a more exhaustive search
# param_grid_xgb = {
#     'regressor__n_estimators': [200, 300],
#     'regressor__learning_rate': [0.05, 0.1],
#     'regressor__max_depth': [5, 7],
# }
# grid_search_xgb = GridSearchCV(xgb_pipeline, param_grid_xgb, cv=3, scoring='r2', n_jobs=-1, verbose=1)
# grid_search_xgb.fit(X_train, y_train)
# best_xgb_model = grid_search_xgb.best_estimator_
# print(f"  > Best XGBoost parameters: {grid_search_xgb.best_params_}")


# # --- Step 6: Create a dictionary of all regression models (tuned and untuned) ---
# # Now we compare the deeply tuned models against the baseline models.
# models = {
#     'Linear Regression': Pipeline(steps=[('preprocessor', preprocessor), ('regressor', LinearRegression())]),
#     'RidgeCV': Pipeline(steps=[('preprocessor', preprocessor), ('regressor', RidgeCV(alphas=np.logspace(-6, 6, 13)))]),
#     'LassoCV': Pipeline(steps=[('preprocessor', preprocessor), ('regressor', LassoCV(alphas=np.logspace(-6, 6, 13), cv=5))]),
#     'ElasticNetCV': Pipeline(steps=[('preprocessor', preprocessor), ('regressor', ElasticNetCV(alphas=np.logspace(-6, 6, 13), cv=5))]),
#     'Random Forest Regressor': Pipeline(steps=[('preprocessor', preprocessor), ('regressor', RandomForestRegressor(random_state=42))]),
#     'Tuned Random Forest Regressor': best_rf_model,
#     'Gradient Boosting Regressor': Pipeline(steps=[('preprocessor', preprocessor), ('regressor', GradientBoostingRegressor(random_state=42))]),
#     'Tuned Gradient Boosting Regressor': best_gb_model,
#     'XGB Regressor': Pipeline(steps=[('preprocessor', preprocessor), ('regressor', XGBRegressor(objective='reg:squarederror', random_state=42))]),
#     'Tuned XGB Regressor': best_xgb_model,
#     'SVR': Pipeline(steps=[('preprocessor', preprocessor), ('regressor', SVR())]),
#     'KNeighbors Regressor': Pipeline(steps=[('preprocessor', preprocessor), ('regressor', KNeighborsRegressor(n_neighbors=5))]),
#     'AdaBoost Regressor': Pipeline(steps=[('preprocessor', preprocessor), ('regressor', AdaBoostRegressor(random_state=42))]),
#     'Decision Tree Regressor': Pipeline(steps=[('preprocessor', preprocessor), ('regressor', DecisionTreeRegressor(random_state=42))]),
#     'LGBM Regressor': Pipeline(steps=[('preprocessor', preprocessor), ('regressor', LGBMRegressor(random_state=42))]),
#     'CatBoost Regressor': Pipeline(steps=[('preprocessor', preprocessor), ('regressor', CatBoostRegressor(verbose=0, random_state=42))])
# }


# # --- Step 7: Train and Evaluate all models in a loop ---
# print("\n--- Training and Evaluating all Regression Models ---")

# # First, get the number of features after preprocessing to calculate Adjusted R2
# X_train_transformed = preprocessor.fit_transform(X_train)
# n_features = X_train_transformed.shape[1]

# for name, model in models.items():
#     print(f"\nTraining {name}...")
#     try:
#         if 'SVR' in name:
#             # Use the subsampled data for SVR
#             model.fit(X_train_svr, y_train_svr)
#             predictions = model.predict(X_test)
#         else:
#             # Use the full training data for all other models
#             model.fit(X_train, y_train)
#             predictions = model.predict(X_test)
        
#         # Calculate all requested metrics
#         mse = mean_squared_error(y_test, predictions)
#         mae = mean_absolute_error(y_test, predictions)
#         rmse = np.sqrt(mse)
#         rmsle_score = rmsle(y_test, predictions)
#         r2 = r2_score(y_test, predictions)
#         adj_r2_score = adj_r2(y_test, predictions, n_features)
#         mape_score = mape(y_test, predictions)
        
#         print(f"--- {name} Results ---")
#         print(f"Mean Absolute Error (MAE): {mae:.2f}")
#         print(f"Mean Squared Error (MSE): {mse:.2f}")
#         print(f"Root Mean Squared Error (RMSE): {rmse:.2f}")
#         print(f"Root Mean Squared Logarithmic Error (RMSLE): {rmsle_score:.4f}")
#         print(f"R-squared (R²): {r2:.4f}")
#         print(f"Adjusted R-squared (Adj_R²): {adj_r2_score:.4f}")
#         print(f"Mean Absolute Percentage Error (MAPE): {mape_score:.2f}%")
        
#     except Exception as e:
#         print(f"An error occurred while training {name}: {e}")

# print("\n--- All models evaluated. ---")


In [ ]:
# Import all necessary libraries
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression, RidgeCV, LassoCV, ElasticNetCV
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, AdaBoostRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neural_network import MLPRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from scipy.stats import randint, uniform

# Assume df_final_features and df1_scaled['duration'] are already defined
# X = df_final_features
# y = df1_scaled['duration']

# --- Step 1: Split data into training and testing sets ---
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# --- Step 2: Create a smaller training sample for SVR ---
X_train_svr, _, y_train_svr, _ = train_test_split(X_train, y_train, test_size=0.9, random_state=42)
print(f"Created a smaller training sample for SVR with {len(X_train_svr)} observations.")

# --- Step 3: Define Preprocessing ---
numeric_features = X.select_dtypes(include=np.number).columns
categorical_features = X.select_dtypes(include='object').columns

numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])


# --- Step 4: Define custom evaluation functions ---
def mape(y_true, y_pred):
    """Calculates the Mean Absolute Percentage Error (MAPE)."""
    epsilon = 1e-10
    return np.mean(np.abs((y_true - y_pred) / (y_true + epsilon))) * 100

def rmsle(y_true, y_pred):
    """Calculates the Root Mean Squared Logarithmic Error (RMSLE)."""
    y_pred[y_pred < 0] = 0
    return np.sqrt(mean_squared_error(np.log1p(y_true), np.log1p(y_pred)))

def adj_r2(y_true, y_pred, n_features):
    """Calculates the Adjusted R-squared."""
    n = y_true.shape[0]
    p = n_features
    r2 = r2_score(y_true, y_pred)
    if p >= n - 1:
        return np.nan
    return 1 - (1 - r2) * (n - 1) / (n - p - 1)


# --- Step 5: Hyperparameter Tuning (Faster Randomized Search) ---
print("\n--- Performing Faster Randomized Hyperparameter Tuning for top models ---")

# Define parameter distributions for RandomizedSearchCV
param_dist_rf = {
    'regressor__n_estimators': randint(100, 500),
    'regressor__max_depth': randint(10, 30),
    'regressor__min_samples_leaf': randint(1, 5)
}

param_dist_gb = {
    'regressor__n_estimators': randint(100, 500),
    'regressor__learning_rate': uniform(0.01, 0.2),
    'regressor__max_depth': randint(3, 10),
}

param_dist_xgb = {
    'regressor__n_estimators': randint(100, 500),
    'regressor__learning_rate': uniform(0.01, 0.2),
    'regressor__max_depth': randint(3, 10),
}

# --- a) Random Forest Regressor (Randomized Search) ---
print("\n  > Starting Randomized Search for Random Forest...")
rf_pipeline = Pipeline(steps=[('preprocessor', preprocessor), ('regressor', RandomForestRegressor(random_state=42))])
random_search_rf = RandomizedSearchCV(rf_pipeline, param_dist_rf, n_iter=50, cv=3, scoring='r2', n_jobs=-1, verbose=1, random_state=42)
random_search_rf.fit(X_train, y_train)
best_rf_model = random_search_rf.best_estimator_
print(f"  > Best Random Forest parameters: {random_search_rf.best_params_}")


# --- b) Gradient Boosting Regressor (Randomized Search) ---
print("\n  > Starting Randomized Search for Gradient Boosting...")
gb_pipeline = Pipeline(steps=[('preprocessor', preprocessor), ('regressor', GradientBoostingRegressor(random_state=42))])
random_search_gb = RandomizedSearchCV(gb_pipeline, param_dist_gb, n_iter=50, cv=3, scoring='r2', n_jobs=-1, verbose=1, random_state=42)
random_search_gb.fit(X_train, y_train)
best_gb_model = random_search_gb.best_estimator_
print(f"  > Best Gradient Boosting parameters: {random_search_gb.best_params_}")


# --- c) XGBoost Regressor (Randomized Search) ---
print("\n  > Starting Randomized Search for XGBoost...")
xgb_pipeline = Pipeline(steps=[('preprocessor', preprocessor), ('regressor', XGBRegressor(random_state=42, objective='reg:squarederror'))])
random_search_xgb = RandomizedSearchCV(xgb_pipeline, param_dist_xgb, n_iter=50, cv=3, scoring='r2', n_jobs=-1, verbose=1, random_state=42)
random_search_xgb.fit(X_train, y_train)
best_xgb_model = random_search_xgb.best_estimator_
print(f"  > Best XGBoost parameters: {random_search_xgb.best_params_}")


# --- Step 6: Create a dictionary of all regression models (tuned and untuned) ---
models = {
    'Linear Regression': Pipeline(steps=[('preprocessor', preprocessor), ('regressor', LinearRegression())]),
    'RidgeCV': Pipeline(steps=[('preprocessor', preprocessor), ('regressor', RidgeCV(alphas=np.logspace(-6, 6, 13)))]),
    'LassoCV': Pipeline(steps=[('preprocessor', preprocessor), ('regressor', LassoCV(alphas=np.logspace(-6, 6, 13), cv=5))]),
    'ElasticNetCV': Pipeline(steps=[('preprocessor', preprocessor), ('regressor', ElasticNetCV(alphas=np.logspace(-6, 6, 13), cv=5))]),
    'Random Forest Regressor': Pipeline(steps=[('preprocessor', preprocessor), ('regressor', RandomForestRegressor(random_state=42))]),
    'Tuned Random Forest Regressor': best_rf_model,
    'Gradient Boosting Regressor': Pipeline(steps=[('preprocessor', preprocessor), ('regressor', GradientBoostingRegressor(random_state=42))]),
    'Tuned Gradient Boosting Regressor': best_gb_model,
    'XGB Regressor': Pipeline(steps=[('preprocessor', preprocessor), ('regressor', XGBRegressor(objective='reg:squarederror', random_state=42))]),
    'Tuned XGB Regressor': best_xgb_model,
    'SVR': Pipeline(steps=[('preprocessor', preprocessor), ('regressor', SVR())]),
    'KNeighbors Regressor': Pipeline(steps=[('preprocessor', preprocessor), ('regressor', KNeighborsRegressor(n_neighbors=5))]),
    'AdaBoost Regressor': Pipeline(steps=[('preprocessor', preprocessor), ('regressor', AdaBoostRegressor(random_state=42))]),
    'Decision Tree Regressor': Pipeline(steps=[('preprocessor', preprocessor), ('regressor', DecisionTreeRegressor(random_state=42))]),
    'LGBM Regressor': Pipeline(steps=[('preprocessor', preprocessor), ('regressor', LGBMRegressor(random_state=42))]),
    'CatBoost Regressor': Pipeline(steps=[('preprocessor', preprocessor), ('regressor', CatBoostRegressor(verbose=0, random_state=42))])
}


# --- Step 7: Train and Evaluate all models in a loop ---
print("\n--- Training and Evaluating all Regression Models ---")

# First, get the number of features after preprocessing to calculate Adjusted R2
X_train_transformed = preprocessor.fit_transform(X_train)
n_features = X_train_transformed.shape[1]

for name, model in models.items():
    print(f"\nTraining {name}...")
    try:
        if 'SVR' in name:
            # Use the subsampled data for SVR
            model.fit(X_train_svr, y_train_svr)
            predictions = model.predict(X_test)
        else:
            # Use the full training data for all other models
            model.fit(X_train, y_train)
            predictions = model.predict(X_test)
        
        # Calculate all requested metrics
        mse = mean_squared_error(y_test, predictions)
        mae = mean_absolute_error(y_test, predictions)
        rmse = np.sqrt(mse)
        rmsle_score = rmsle(y_test, predictions)
        r2 = r2_score(y_test, predictions)
        adj_r2_score = adj_r2(y_test, predictions, n_features)
        mape_score = mape(y_test, predictions)
        
        print(f"--- {name} Results ---")
        print(f"Mean Absolute Error (MAE): {mae:.2f}")
        print(f"Mean Squared Error (MSE): {mse:.2f}")
        print(f"Root Mean Squared Error (RMSE): {rmse:.2f}")
        print(f"Root Mean Squared Logarithmic Error (RMSLE): {rmsle_score:.4f}")
        print(f"R-squared (R²): {r2:.4f}")
        print(f"Adjusted R-squared (Adj_R²): {adj_r2_score:.4f}")
        print(f"Mean Absolute Percentage Error (MAPE): {mape_score:.2f}%")
        
    except Exception as e:
        print(f"An error occurred while training {name}: {e}")

print("\n--- All models evaluated. ---")


In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

# --- 1. Create a sample dataset for demonstration ---
# In your actual project, you would load your cleaned and processed X and y data here.
# X_processed should be your feature-selected, scaled, and encoded training data.
# y should be your target variable.
np.random.seed(42)
X = np.random.rand(100, 10) * 100
y = (X[:, 0] > 50).astype(int)  # A simple binary target for demonstration purposes

# --- 2. Split the data into training and testing sets ---
# It is crucial to split the data *before* tuning to prevent data leakage.
# Grid search will be performed only on the training set.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# --- 3. Define the model you want to tune ---
# This is where you would place your individual model instance.
# For this example, we use a Support Vector Classifier (SVC).
model = SVC()

# --- 4. Define the hyperparameter grid ---
# This dictionary specifies the parameters to tune and the values to test for each.
# For SVC, we will tune the 'C' and 'kernel' parameters.
# 'C' is a regularization parameter.
# 'kernel' is the function used to map data into a higher-dimensional space.
param_grid = {
    'C': [0.1, 1, 10, 100],
    'kernel': ['linear', 'rbf']
}

# --- 5. Set up and run the Grid Search ---
# GridSearchCV will train a model for every combination of parameters in the grid.
# cv=5 means it will use 5-fold cross-validation on the training data.
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=5, scoring='accuracy', verbose=1, n_jobs=-1)

# Fit the grid search to the training data.
# This will perform the exhaustive search for the best parameters.
print("Starting Grid Search...")
grid_search.fit(X_train, y_train)

# --- 6. Review the results ---
print("\n--- Tuning Results ---")
print(f"Best parameters found: {grid_search.best_params_}")
print(f"Best cross-validation accuracy: {grid_search.best_score_:.4f}")

# --- 7. Evaluate the best model on the test data ---
# We use the best model found by the grid search and apply it to the unseen test data.
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)
test_accuracy = accuracy_score(y_test, y_pred)

print(f"Accuracy of the best model on the test set: {test_accuracy:.4f}")
